# 导入函数库

In [1]:
import tensorflow as tf
import tempfile
import zipfile
import os
import numpy as np
import tensorflow_model_optimization
from tensorflow_model_optimization.sparsity import keras as sparsity

# 加载训练集和测试集

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

## 把数据集的INT8转换成Float32类型，对标签进行独热编码

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255  # 缩放到[0,1]
x_test /= 255
print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10  # cifar10属于10分类问题
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

x_train shape:  (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 展示编码后的标签

In [4]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

# 构建VGG16模型，用于cifar10分类，代码参考自keras github的VGG16

## 构建模型的一些参数设置

In [5]:
backend = tf.keras.backend
layers = tf.keras.layers
models = tf.keras.models
keras_utils = tf.keras.utils

In [6]:
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.1/'
                       'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

## 构建VGG16模型结构

In [7]:
def VGG16(include_top=False,
         weights='imagenet',
         input_tensor=None,
         input_shape=(32, 32, 3),
         pooling='avg',
         classes=10,
         **kwargs):
    img_input = layers.Input(shape=input_shape)  # 输入节点
    
    # Block 1
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    if pooling == 'avg':
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(128, activation='relu', name='fc1')(x)
        x = layers.Dense(classes, activation='softmax', name='predictions')(x) # 此处是10分类，而不是ImageNet的1000分类
    
    inputs = img_input  # 输入节点, x是输出节点
    model = models.Model(inputs, x , name='vgg16')  # 生成一个Model, 需要指定输入和输出
    
    weights_path = keras_utils.get_file(
        'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',  # 此处加载的是不存在全连接层的预训练模型
        WEIGHTS_PATH_NO_TOP,
        cache_subdir='models',
        file_hash='6d6bbae143d832006294945121d1f1fc')
    model.load_weights(weights_path, by_name=True)  # 加载在ImageNet上预训练过的模型，注意by_name参数很有用
    
    return model

## 实例化一个vgg16对象

In [8]:
vgg16 = VGG16()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## 查看模型的结构

In [9]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

# 对模型进行剪枝

## 设置pruning参数

In [10]:
epochs = 10
batch_size = 128
num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

End step: 3910


In [11]:
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.5,  # 起始稀疏度
                                                final_sparsity=0.9,  # 最终稀疏度
                                                begin_step=2000,  # 从哪一个step开始剪枝
                                                end_step=end_step,  # 剪枝结束的step
                                                frequency=100)  # 每100个step进行剪枝一次
}

## 得到剪枝后的vgg16模型

In [12]:
vgg16_pruned = sparsity.prune_low_magnitude(vgg16, **pruning_params)

Instructions for updating:
Please use `layer.add_weight` method instead.

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



## 查看剪枝后的模型结构

In [13]:
vgg16_pruned.summary()  # 多了一层pruning wrapper

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
prune_low_magnitude_block1_c (None, 32, 32, 64)        3522      
_________________________________________________________________
prune_low_magnitude_block1_c (None, 32, 32, 64)        73794     
_________________________________________________________________
prune_low_magnitude_block1_p (None, 16, 16, 64)        1         
_________________________________________________________________
prune_low_magnitude_block2_c (None, 16, 16, 128)       147586    
_________________________________________________________________
prune_low_magnitude_block2_c (None, 16, 16, 128)       295042    
_________________________________________________________________
prune_low_magnitude_block2_p (None, 8, 8, 128)         1     

# 模型训练

## 设置Logdir

In [14]:
logdir = tempfile.mkdtemp()
print('writing training logs to ' + logdir)

writing training logs to /tmp/tmpekr9znv8


## compile模型

In [15]:
vgg16_pruned.compile(
    loss = tf.keras.losses.categorical_crossentropy, # 分类模型一般用这个损失函数
    optimizer = 'adam',  # 采用的优化器
    metrics = ['accuracy']
)

## 设置callbacks

In [16]:
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

## 开始训练模型

In [17]:
vgg16_pruned.fit(x_train, y_train,
                batch_size = 128,
                epochs = epochs,
                verbose=1,
                callbacks = callbacks,
                validation_data = (x_test, y_test)
                )

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 33s 663us/sample - loss: 1.9306 - acc: 0.2475 - val_loss: 1.5411 - val_acc: 0.3762
Epoch 2/10
50000/50000 [==============================] - 26s 514us/sample - loss: 1.2603 - acc: 0.5275 - val_loss: 1.0206 - val_acc: 0.6378
Epoch 3/10
50000/50000 [==============================] - 26s 512us/sample - loss: 0.9266 - acc: 0.6754 - val_loss: 0.8231 - val_acc: 0.7164
Epoch 4/10
50000/50000 [==============================] - 26s 512us/sample - loss: 0.7328 - acc: 0.7489 - val_loss: 0.7085 - val_acc: 0.7675
Epoch 5/10
50000/50000 [==============================] - 26s 513us/sample - loss: 0.6112 - acc: 0.7946 - val_loss: 0.6920 - val_acc: 0.7741
Epoch 6/10
50000/50000 [==============================] - 26s 526us/sample - loss: 0.4732 - acc: 0.8448 - val_loss: 0.6068 - val_acc: 0.8010
Epoch 7/10
50000/50000 [==============================] - 26s 524us/sample - loss: 0.4095 - acc: 0.8643 

## 评估模型损失值和正确率

In [18]:
score = vgg16_pruned.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  0.629626203250885
Test accuracy:  0.8118


## 对模型去除pruning wrapper

In [19]:
final_model = sparsity.strip_pruning(vgg16_pruned)

# 模型保存

In [20]:
_, pruned_keras_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', pruned_keras_file)
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

Saving pruned model to:  /tmp/tmp31d0lw57.h5


## 对剪枝模型进行压缩，对未剪枝模型进行压缩，对比两者大小

In [21]:
_, zip3 = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip3, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" 
      % (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" 
      % (os.path.getsize(zip3) / float(2**20)))

Size of the pruned model before compression: 56.45 Mb
Size of the pruned model after compression: 10.85 Mb


# 转化成FP32的.tflite

In [22]:
tflite_model_file = './sparse_cifar10.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file) # 获得converter对象
tflite_model = converter.convert()  # 进行转换
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.


# 封装TFLite模型准确率验证函数

In [23]:
def eval_model(interpreter, x_test, y_test):
    total_seen = 0
    num_correct = 0
    
    for img, label in zip(x_test, y_test):
        inp = img.reshape((1, 32, 32, 3))
        total_seen += 1
        interpreter.set_tensor(input_index, inp)  # 获得输入
        interpreter.invoke()  # 执行推断
        predictions = interpreter.get_tensor(output_index) # 获取输出
        
        if np.argmax(predictions) == np.argmax(label):
            num_correct += 1
        
        if total_seen % 1000 == 0:
            print('Accuracy after %i images: %f' % (total_seen, float(num_correct) / float(total_seen)))
    
    return float(num_correct) / float(total_seen)

# 转化成FP32的.tflite，我们发现模型没什么变化，只是存储格式发生了变化

In [24]:
_, zip_tflite = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 56.39 Mb
Size of the tflite model after compression: 10.77 Mb


## 验证FP32的.tflite模型准确率

In [25]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))  # 获取interpreter对象
interpreter.allocate_tensors() # 为tensor分配内存
input_index = interpreter.get_input_details()[0]["index"] # 获取输入
output_index = interpreter.get_output_details()[0]["index"]  # 获取输出

eval_model(interpreter, x_test, y_test)  # 执行评估函数

Accuracy after 1000 images: 0.819000
Accuracy after 2000 images: 0.813500
Accuracy after 3000 images: 0.809333
Accuracy after 4000 images: 0.810250
Accuracy after 5000 images: 0.811600
Accuracy after 6000 images: 0.813833
Accuracy after 7000 images: 0.812714
Accuracy after 8000 images: 0.812625
Accuracy after 9000 images: 0.812778
Accuracy after 10000 images: 0.811800


0.8118

# 转化过程中引入weight_quantization

In [26]:
tflite_quant_model_file = './sparse_cifar10_quant.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file) # 获得converter对象
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert() # 进行转换
with open(tflite_quant_model_file, 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.


## 对比weight_quantization前后模型体积变化

In [27]:
_, zip_tflite = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_quant_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 14.12 Mb
Size of the tflite model after compression: 2.54 Mb


## 验证tflite模型准确率,查看weight quantization后，下降了多少

In [28]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_file)) # 获得interpreter对象
interpreter.allocate_tensors() # 为tensor分配内存
input_index = interpreter.get_input_details()[0]["index"]  # 获取输入
output_index = interpreter.get_output_details()[0]["index"]  #  获取输出

eval_model(interpreter, x_test, y_test)  # 执行评估函数

Accuracy after 1000 images: 0.821000
Accuracy after 2000 images: 0.812000
Accuracy after 3000 images: 0.809667
Accuracy after 4000 images: 0.811250
Accuracy after 5000 images: 0.812200
Accuracy after 6000 images: 0.814333
Accuracy after 7000 images: 0.813286
Accuracy after 8000 images: 0.813000
Accuracy after 9000 images: 0.812444
Accuracy after 10000 images: 0.811800


0.8118